<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/Unidad_3/2_Divide_y_venceras_con_estructuras_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

En esta sesión, veremos dos estructuras de datos que están muy ligadas a las ideas de los algoritmos de divide y vencerás, que suelen ser muy útiles para realizar operaciones que suelen requerirse bastante.

# Trie

Consideremos el siguiente problema. Dadas dos palabras, decimos que su "similitud" es el mayor número de letras iniciales en las que coinciden, es decir, es el tamaño del prefijo común más grande entre ambas palabras. Dado un conjunto de palabras $C$, ,y una nueva palabra, $s$, queremos saber la mayor similitud posible con alguna letra del conjunto. 

¿Cómo proceder? Una forma de hacerlo es haciendo una búsqueda exhaustiva, donde para cada palabra de $C$ la comparamos con la palabra $s$, y vamos verificando letra por letra los prefijos hasta que dejen de coincidir. Podemos acotar la complejidad en tiempo por $O(|C|\cdot |s|)$. ¿Qué pasa si queremos repetir el proceso para una cantidad muy grande de palabras s?

Otra forma de atacar el problema sería ordenar las palabras en orden alfabético, incluyendo a $s$, y posteriormente comparar a $s$ con las palabras vecinas en el ordenamiento (queda como ejercicio probar que en efecto uno de sus vecinos es la palabra que cumple tener mayor similitud con $s$). ¿Qué complejidad tiene esto?

La forma que veremos introduce una estructura, que resulta ser muy poderosa para este tipo de problemas, es un árbol especial, que tiene por nombre *trie*. Un trie es un árbol donde cada nodo puede tener $26$ hijos (uno por cada letra del abecedario). De este modo, si una palabra tiene las letras "xy" de forma consecutiva, el respectivo nodo tendrá un hijo no nulo en la entrada correspondiente a "y". Veamos un ejemplo de esta estructura.

Consideremos las palabras `[abcdef, abecede, hola, hoja, puedes, luz]`. El trie respectivo a este conjunto de palabras es el siguiente:

```
       (Nodo raíz)
      /   |   |  \
     a    h   p   l
     |    |   \    \
     b    o    u    u
    / \   | \   \    \
   c   e  l  j   e    z
   |   |  |  |   |
   d   c  a  a   d
   |   |         |
   e   e         e
   |   |         |
   f   d         s
       |
       e
```

Esta estructura nos permite identificar de manera rápida y sencilla el mayor prefijo que coincide con cierta palabra. Veamos cómo implementar esta estructura y cómo usarla para nuestro problema.

In [2]:
x = None

In [3]:
def char_index(c):
    return ord(c) - ord('a')

class TrieNode:
    def __init__(self):
        self.children = [None]*26
        
    def insert(self, word):
        word = word.casefold()
        current = self
        for c in word:
            idx = char_index(c)
            if current.children[idx] is None:
                current.children[idx] = TrieNode()
            current = current.children[idx]
            
    def count_prefix(self, word):
        curr = self
        count = 0
        for c in word:
            idx = char_index(c)
            if curr.children[idx] is None:
                return count
            else:
                curr = curr.children[idx]
                count += 1
        return count


C = ["abcdef", "abecede", "hola", "hoja", "puedes", "luz"]
root = TrieNode()
for c in C:
    root.insert(c)

S = ["abaco", "holandes", "paz"]
for s in S:
    print(root.count_prefix(s))

2
4
1


# Sumas de intervalos

Dada una lista $L$ de números reales ($|L| = n$), queremos saber la suma de los elementos en un intervalo dado, es decir $L_i + L_{i+1} + \dots + L_j$ para determinados $i \leq j$. Además de esto, queremos poder actualizar nuestra lista $L$. Es decir, tenemos dos tipos de operaciones:

*   Preguntar por la suma de los valores con índices en el intervalo $[i,j]$.
*   Actualizar el valor de cierto elemento $L_i$.

Una forma sencilla de poder recuperar el valor de la suma en cierto intervalo es guardando una lista de sumas acumuladas, sin embargo, esto hace que las operaciones de actualización de elementos sean costosas.

## Bloques

¿Qué pasa si dividimos la lista en sublistas de longitud $k$? (Veremos después qué $k$ nos conviene). Tendremos un total de $\left\lceil\frac{n}{k}\right\rceil$ 'bloques', para cada bloque guardaremos el valor de la suma de sus elementos. ¿Qué tan costosas son las operaciones de actualización y preguntar por sumas? 

* Para actualizar, únicamente debemos actualizar el valor del elemento, así como la suma correspondiente en el bloque al que pertenece, por lo que cada actualización nos toma $O(1)$. 
* Para calcular la suma de un intervalo $I$, notamos que para cada bloque $B$ existen tres casos:

    1. $B\cap I=\emptyset$. En este caso, no hacemos nada. ($O(1)$)
    2. $B\subseteq I$. Aquí, simplemente añadimos la suma del bloque entero (que tenemos guardada). ($O(1)$)
    3. $B\cap I \neq \emptyset \wedge B\not\subseteq I$. En este caso, tenemos que iterar sobre $B$ para encontrar cuáles son los elementos que están en $I$, y sumarlos. ($O(k)$)
    
El tercer caso sólo puede pasar para dos bloques: los que están en el extremo izquierdo y derecho del intervalo. Por lo tanto, la complejidad de estos casos es $O(k)$. 

Por otro lado, como hay un total de $\frac{n}{k}$ bloques, el primer y segundo caso no pueden pasar más de esta cantidad, así que su complejidad es $(O(\frac{n}{k}))$.

Por lo tanto, la complejidad del problema completo es $O\left(\frac{n}{k}+k\right)$.

Entonces, queremos encontrar la $k$ que minimice esta cantidad. Derivando e igualando a cero, obtenemos $k=\sqrt{n}$.

## Segment tree

Otra manera de resolver este problema es utilizar un árbol binario, donde cada nodo representa a un intervalo, y tiene los siguientes atributos:

* `value`: Suma del intervalo.
* `left`: Hijo izquierdo.
* `right`: Hijo derecho.
* `start`: Extremo izquierdo del intervalo.
* `end`: Extremo derecho del intervalo.

El intervalo de un nodo padre es la unión de los intervalos de sus dos hijos. Los nodos hoja corresponden a los intervalos triviales de la lista original (i.e. $[L_0, L_0], [L_1, L_1]$, etc.)

De este modo, logramos que preguntar por la suma en un intervalo $[l,r]$ sea $O(logn)$, mientras que actualizar también nos tomará $O(logn)$, ya que esta es la altura de nuestro árbol.

Veamos una implementación de esta estructura y un ejemplo de su uso.

In [45]:
class SegNode:
    def __init__(self, L, l=None, r=None):
        self.value = 0
        
        if l is None:
            l = 0
        if r is None:
            r = len(L) - 1
        
        self.start = l
        self.end = r
        if self.start == self.end:
            self.value = L[self.start]
            self.left = None
            self.right = None
        else:
            m = (l+r)//2
            self.left = SegNode(L, l=self.start, r=m)
            self.right = SegNode(L, l=m+1, r=self.end)
            self.value += self.left.value
            self.value += self.right.value
    
    def _arr_str(self):
        return f'[{self.start}, {self.end}]'
    
    def __repr__(self):
        s1 = s2 = ''
        if self.left is not None:
            s1 = self.left._arr_str()
        if self.right is not None:
            s2 = self.right._arr_str()
        return f'{self._arr_str()}\nHijos: I: {s1}, D:{s2}'
    
    def query(self, l, r):
        if l <= self.start and self.end <= r:
            return self.value
        if self.end < l:
            return 0
        if r < self.start:
            return 0
        
        s = 0
        s += self.left.query(l, r) + self.right.query(l, r)
        return s
    
    def update(self, i, val):
        if self.end < i or i < self.start:
            return self.value
        if self.end == self.start == i:
            self.value = val
        else:
            self.value = self.left.update(i, val) + self.right.update(i, val)
        return self.value

Generamos una lista aleatoria:

In [93]:
L = np.random.randint(-20, 20, 100)
L

array([  0,  -6, -18, -19,   7, -19,  15,  -8,  -4,  10,   2,  -9,  -1,
        17,   5, -14,  -7,  -3, -18,  18, -17,  -7, -15,   9,   8,  11,
        17,  -3, -16,   7, -10,  12,   2, -18,  -5,   1,  -6, -18, -18,
        -1,   4,   7,  -5,   6, -16,   4,  15,   7,  18,  18, -15,  -3,
        13,   8,  -6,  -8,   1,  14,  15,  16,   9,   8,  18,   1,  -8,
        -9,   5,   8,   4, -20,  -4,  10,  17,   1,   7, -15,   9, -19,
       -15, -12,   9,   2,  14,  10,   8,   7,   2,  19,  -1, -12,  -6,
        11,   9,  19, -14, -10,  12,   2,  -8,   1])

Creamos el árbol de segmentos:

In [94]:
root = SegNode(L)

Calculamos algunas sumas:

In [98]:
l, r = 0, 99
print(f"Suma desde el índice {l} hasta {r}: {root.query(l, r)}")

l, r = 5, 7
print(f"Suma desde el índice {l} hasta {r}: {root.query(l, r)}")

l, r = 8, 8
print(f"Suma desde el índice {l} hasta {r}: {root.query(l, r)}")

Suma desde el índice 0 hasta 99: 43
Suma desde el índice 5 hasta 7: -12
Suma desde el índice 8 hasta 8: -4


Sustituimos el valor de la primera entrada:

In [99]:
root.update(0, 100)

143

Y volvemos a calcular:

In [100]:
l, r = 0, 99
print(f"Suma desde el índice {l} hasta {r}: {root.query(l, r)}")

l, r = 5, 7
print(f"Suma desde el índice {l} hasta {r}: {root.query(l, r)}")

l, r = 8, 8
print(f"Suma desde el índice {l} hasta {r}: {root.query(l, r)}")

Suma desde el índice 0 hasta 99: 143
Suma desde el índice 5 hasta 7: -12
Suma desde el índice 8 hasta 8: -4


# Ejercicios

## Ejercicio 1

Extiende la clase de Trie para determinar si una palabra o no está en el conjunto de palabras dado (*Hint:* Añade un atributo a los nodos que diga si representa el final de una palabra). Prueba tu código con el conjunto $C = \{ola, hola, noche, diez, no, ciencia, camaron\}$ y las palabras $cama, hola, nop, cien, diez$.

In [ ]:
# Aquí va tu código

## Ejercicio 2

En la siguiente celda se muestra un prototipo de una clase para implementar el método de bloques. Su constructor debe de tomar una lista `L` arbitraria. Como ejemplo, tomaremos `L=[1,3,5,7]`. Con esto, los atributos que debe de tener son:

* `k`: Tamaño de cada bloque, de modo que sea óptimo. En el ejemplo, `k=2`.
* `blocks`: Lista de todos los bloques. Cada bloque es a su vez una lista. Para la lista de ejemplo, esto sería igual a `[[1,3],[5,7]]`.
* `sums`: Lista con las sumas de todos los elementos de cada bloque. En el ejemplo, sería `[4, 12]`.

Por otro lado, sus métodos son:

* `query(l, r)`: Suma de los elementos desde el índice `l` hasta `r`, inclusivo. Tomando nuestra lista de ejemplo, `query(0, 2) = 9`.
* `update(i, val)`: Actualiza el valor del elemento en el índice `i`, con el nuevo valor `val`. Debe de actualizar también el bloque y la suma correspondiente.

Reemplaza todos los `None` en el siguiente ejemplo con el código apropiado. Los métodos deben de aplicar lo que vimos en clase (¡nada de iterar sobre la lista entera cada vez que quieras calcular la suma!).

In [88]:
class Blocks:
    def __init__(self, L): # L: lista a procesar
        self.k = None 
        self.blocks = None
        self.sums = None 
        
    def query(self, l, r):
        None
        
    def update(self, i, val):
        None

Después, ejecuta la siguiente celda (sin cambiar nada), y compara tus resultados:

In [89]:
L = [5, -11, 9, 3, 10, 2, -2, 18, -12, -9, -19, 11, -18, -1, -13, -14, 19, -15, 1, -19, 15, 19, 9, -1, -13, -6, 4, -19, -9, 1, 8, -20, 11, 2, 8, -20, -19, -2, -2, 1, 1, 12, -16, -8, -9, 11, -3, 10, 15, -6, 9, 8, -5, 7, -12, -1, -16, -7, 7, 16, 5, 11, -11, -11, 8, 18, -6, 1, -19, 17, -18, 15, 5, -18, 7, -5, -13, 19, -6, -19, -15, 4, 10, 19, -2, 18, -7, 11, -14, -4, -10, -18, 4, 14, -18, -3, -19, -8, 7, -1]
blocks = Blocks(L)

r = blocks.query(0, 99)
print(f"Resultado esperado: -127. Obtenido: {r}")

r = blocks.query(5, 10)
print(f"Resultado esperado: -22. Obtenido: {r}")

blocks.update(1, 100)

r = blocks.query(0, 99)
print(f"Resultado esperado: -16. Obtenido: {r}")

r = blocks.query(5, 10)
print(f"Resultado esperado: -22. Obtenido: {r}")

Resultado esperado: -127. Obtenido: None
Resultado esperado: -22. Obtenido: None
Resultado esperado: -16. Obtenido: None
Resultado esperado: -22. Obtenido: None
